In [6]:
!pip install influxdb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 3.6 MB/s eta 0:00:00ta 0:00:01


In [9]:
import csv

In [11]:
ls

 Análisis_de_logs_con_MapReduce.ipynb
 clean_file_bueno.csv
 Cola_de_pedidos_encapsulada_en_funciones.ipynb
 conexion_influx.ipynb*
 data/
'Datos geoespaciales.ipynb'
 Escritura_de_datos_en_InfluxDB.ipynb*
 Examen_propio_Influxdb.ipynb
 Examen_propio_redis.ipynb
'Hashes, sorted sets e hyperloglogs.ipynb'
 index.md*
 Ingesta_de_datos_financieros_en_InfluxDB.ipynb
 locations.py
 mezcla_prueba.ipynb
 Prueba_examen.ipynb
 __pycache__/
 redis.ipynb
'Trabajo con Python.ipynb'
 Untitled.ipynb


In [15]:
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS, WriteOptions
from influxdb_client.client.exceptions import InfluxDBError
from urllib3.exceptions import NewConnectionError
from influxdb_client import Point

INFLUX_URL = "http://influxdb-influxdb2-1:8086"
INFLUX_TOKEN = "MyInitialAdminToken0="

print("--- Iniciando conexión a InfluxDB ---")

client = None

try:
    client = influxdb_client.InfluxDBClient(
        url=INFLUX_URL,
        token=INFLUX_TOKEN,
        org="docs"
        )
    print(f"Verificando estado de salud de InfluxDB en {INFLUX_URL}")
    health = client.health()
    
    if health.status == "pass":
        print("[INFO] ¡Conexión Exitosa!")
        print(f"[INFO] Versión del servidor: {health.version}")
        ##Aqui va la logica
        #write_api = client.write_api( 
        #write_options=WriteOptions( 
        #batch_size=1000, #Tamaño del lote 
        #flush_interval=5000, #Tiempo de espera en caso de que no llegue al tamaño del lote 
        #) )
        write_api = client.write_api(write_options=SYNCHRONOUS)
        with open("./clean_file_bueno.csv", "r") as f:
            lector = csv.DictReader(f)
            next(lector) #Saltar cabecera
            for fila in lector:
                #Recorremos las filas en forma de Diccionario
                from datetime import datetime
                fecha_original = fila["date_x"].strip()
                fecha_obj = datetime.strptime(fecha_original, "%m/%d/%Y")
                fecha_iso = fecha_obj.isoformat()
                p = Point("movie") \
                     .tag("names",fila["names"]) \
                     .tag("orig_lang",fila["orig_lang"]) \
                     .tag("country",fila["country"]) \
                     .field("score",float(fila["score"])) \
                     .time(fecha_iso)
                write_api.write(bucket="peliculas",org="docs", record=p) 
                
    else:
        print(f"[Error] Conexion fallida. Estado: {health.status}")
        print(f"[INFO] Mensaje: {health.message}")
except (InfluxDBError,NewConnectionError) as e:
    print("[ERROR] Error al conectar con InfluxDB:")
    print(f" Detalle: {e}")
finally:
    if client:
        client.close()
        print("--- Conexión Cerrada ---")

--- Iniciando conexión a InfluxDB ---
Verificando estado de salud de InfluxDB en http://influxdb-influxdb2-1:8086
[INFO] ¡Conexión Exitosa!
[INFO] Versión del servidor: v2.7.12
--- Conexión Cerrada ---
